## Easily reproduce somewhere else

In [ ]:
%load_ext dotenv
%dotenv

### from cache

In [1]:
! pipenv run dvc repro

Loading .env environment variables...
If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Stage 'data/data.xml.dvc' didn't change, skipping
Stage 'prepare' didn't change, skipping
Stage 'featurize' didn't change, skipping
Stage 'train' didn't change, skipping
Stage 'evaluate' didn't change, skipping
Data and pipelines are up to date.


### or force actual pipeline run

In [4]:
! dvc repro -f

Loading .env environment variables...
If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Verifying data sources in stage: 'data/data.xml.dvc'

Running stage 'prepare' with command:
	python src/prepare.py data/data.xml
  0% Saving prepared|                          |0.00/2.00 [00:00<?,     ?file/s]
Running stage 'featurize' with command:
	python src/featurization.py data/prepared data/features
The input data frame data/prepared/train.tsv size is (20017, 3)
The output matrix data/features/train.pkl size is (20017, 502) and data type is float64
The input data frame data/prepared/test.tsv size is (4983, 3)
The output matrix data/features/test.pkl size is (4983, 502) and data type is float64
  0% Saving features|                          |0.00/2.00 [00:00<?,     ?file/s]
Running stage 'train' with command:
	python src/train.py data/features model.pkl
Input matrix size (20017, 502)
X matrix size (20017, 500)
Y matrix size (20017,)
If DVC froze, see `hardlink_lock` in <http

In [16]:
! dvc diff

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Modified:
    data/features/
    model.pkl
    prc.json
    scores.json

files summary: 0 added, 0 deleted, 3 modified


### Assert equivalent result

In [6]:
! cat scores.json

{"auc": 0.5417487597055675}

### Now I can easily
* Making updates to the models
* see differences in parameters or results
* collaborate with others
* reproduce results

### Go and modify some parameters

In [8]:
! dvc status

featurize:
	changed deps:
		params.yaml:
			modified:           featurize.max_features
train:
	changed deps:
		params.yaml:
			modified:           train.n_estimators


In [6]:
! dvc params diff

Path         Param                   Old    New
params.yaml  featurize.max_features  500    1500
params.yaml  train.n_estimators      50     100


In [9]:
! dvc repro

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Stage 'data/data.xml.dvc' didn't change, skipping
Stage 'prepare' didn't change, skipping
Running stage 'featurize' with command:
	python src/featurization.py data/prepared data/features
The input data frame data/prepared/train.tsv size is (20017, 3)
The output matrix data/features/train.pkl size is (20017, 1502) and data type is float64
The input data frame data/prepared/test.tsv size is (4983, 3)
The output matrix data/features/test.pkl size is (4983, 1502) and data type is float64
  0% Saving features|                          |0.00/2.00 [00:00<?,     ?file/s]Updating lock file 'dvc.lock'

Running stage 'train' with command:
	python src/train.py data/features model.pkl
Input matrix size (20017, 1502)
X matrix size (20017, 1500)
Y matrix size (20017,)
If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Updating lock file 'dvc.lock'

Running stage 'evaluate' with command:
	python src/evaluate.py mo

In [10]:
! dvc metrics diff

Path         Metric    Value    Change
scores.json  auc       0.56925  0.0275


In [13]:
! cat scores.json

{"auc": 0.5692532520676357}

In [12]:
! dvc plots diff -x recall -y precision

file:///home/nichita/Projects/dvc_intro/plots.html
